# KMS on a Confidential Container on ACI

### Step 1 : Setup 

In this step we will 
- Setup the necessary libraries and customizable variables 
- Request the public key from attestation well known endpoint and create jwks object that will be used to verify the attestation tokens later in the demo. 

In [14]:

import subprocess
import json
import base64
import requests
from jose import jwt, JWTError
import json
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography import x509
from hashlib import sha256


registry_name = 'docker.io/tanaybaswa' 
attestation_endpoint = 'sharedeus2.eus2.test.attest.azure.net'

# Random for now, can be updated for specific functionality.
runtime_data = 'eyJrZXlzIjpbeyJlIjoiQVFBQiIsImtleV9vcHMiOlsiZW5jcnlwdCJdLCJraWQiOiJOdmhmdXEyY0NJT0FCOFhSNFhpOVByME5QXzlDZU16V1FHdFdfSEFMel93Iiwia3R5IjoiUlNBIiwibiI6InY5NjVTUm15cDh6Ykc1ZU5GdURDbW1pU2VhSHB1akcyYkNfa2VMU3V6dkRNTE8xV3lyVUp2ZWFhNWJ6TW9PMHBBNDZwWGttYnFIaXNvelZ6cGlORExDbzZkM3o0VHJHTWVGUGYyQVBJTXUtUlNyek41NnF2SFZ5SXI1Y2FXZkhXay1GTVJEd0FlZnlOWVJIa2RZWWtnbUZLNDRoaFVkdGxDQUtFdjVVUXBGWmp2aDRpSTlqVkJkR1lNeUJhS1FMaGpJNVdJaC1RRzZaYTVzU3VPQ0ZNbm11eXV2TjVEZmxwTEZ6NTk1U3MtRW9CSVktTmlsNmxDdHZjR2dSLUlialVZSEFPczVhamFtVHpnZU84a3gzVkNFOUhjeUtteVVac2l5aUY2SURScDJCcHkzTkhUakl6N3Rta3BUSHg3dEhuUnRsZkUyRlV2MEI2aV9RWWxfWkE1USJ9XX0='

# Extract the public key from the openid-configuration and create a JWKS object

def rsa_public_key_from_pem(cert_pem):
    cert = x509.load_pem_x509_certificate(cert_pem.encode(), default_backend())
    return cert.public_key()

response = requests.get(f"https://{attestation_endpoint}/certs")

if response.status_code == 200:
    cert_data = response.json()
    keys = cert_data['keys']

    # Step 2: Create a JWKS object
    jwks = []

    for key_data in keys:
        x5c = key_data.get('x5c', [])
        if x5c:
            cert_pem = "-----BEGIN CERTIFICATE-----\n" + x5c[0] + "\n-----END CERTIFICATE-----"
            public_key = rsa_public_key_from_pem(cert_pem)
            jwks.append((key_data['kid'], public_key))

    print("JWKS object created successfully.")
else:
    print("Failed to retrieve the signing keys.")

JWKS object created successfully.


### Step 2a : Build and upload container image to registry 

In this step we are going to build the "sum" container image that exposes a rest endpoint to calculate the sum of two numbers and also exposes an endpoint to get a MAA attestation token. 

In [9]:
# build and push the image to the registry
subprocess.run(["docker build -t tanaybaswa/attestedkms:latest /home/tanaybaswa/code/enkryptai/key-manager/"], shell=True, capture_output=True, text=True)
subprocess.run(["docker push tanaybaswa/attestedkms:latest"], shell=True, capture_output=True, text=True)

CompletedProcess(args=['docker push tanaybaswa/attestedkms:latest'], returncode=0, stdout='The push refers to repository [docker.io/tanaybaswa/attestedkms]\n5f70bf18a086: Preparing\nf73ab0be9222: Preparing\nf2b087a0b61e: Preparing\nbd6143418689: Preparing\n1847aa24cd34: Preparing\n2f6e140f1970: Preparing\nf30c9841a094: Preparing\na886fdc1df20: Preparing\nb2e5b1eee192: Preparing\nb485c6cd33a6: Preparing\n6aa872026017: Preparing\n43ba18a5eaf8: Preparing\n2f6e140f1970: Waiting\nf30c9841a094: Waiting\na886fdc1df20: Waiting\nff61a9b258e5: Preparing\nb2e5b1eee192: Waiting\nb485c6cd33a6: Waiting\n6aa872026017: Waiting\n43ba18a5eaf8: Waiting\nff61a9b258e5: Waiting\nbd6143418689: Layer already exists\n5f70bf18a086: Layer already exists\nf73ab0be9222: Layer already exists\nf2b087a0b61e: Layer already exists\n1847aa24cd34: Layer already exists\nf30c9841a094: Layer already exists\n2f6e140f1970: Layer already exists\na886fdc1df20: Layer already exists\nb485c6cd33a6: Layer already exists\nb2e5b1eee1

### Step 2b: Convert both the SKR and KMS images into one tar file.

In [10]:
subprocess.run(["docker save tanaybaswa/skr:latest tanaybaswa/attestedkms:latest -o attested_kms.tar"], shell=True, capture_output=True, text=True)


CompletedProcess(args=['docker save tanaybaswa/skr:latest tanaybaswa/attestedkms:latest -o attested_kms.tar'], returncode=0, stdout='', stderr='')

### Step 3 : Generate security policy for sum container 

We will use the confcom tooling to generate a security policy from the Azure Resource Manager template. We will further generate a SHA-256 hash of the security policy which will be used later in the demo to verify whether the container group is running the right configuration. 

Note : The "ccePolicy" attribute of the ARM template must be set to a null string "" for this step to work. The tooling requires user input to override the policy if already present and user input is not supported in the notebook. 

In [15]:
# generate security policy
subprocess.run(["az confcom acipolicygen -a template_kms_skr.json --tar attested_kms.tar"], capture_output=True, shell=True)


CompletedProcess(args=['az confcom acipolicygen -a template_kms_skr.json --tar attested_kms.tar'], returncode=0, stdout=b'708927e5febd5666d4cac83e7cb76b2313c8b85e99626005e1a981486c3aa34f\n', stderr=b'WARNING: Command group \'confcom\' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus\nWARNING: Generating security policy for ARM Template: template_kms_skr.json in base64\n\rPulling and hashing images...:   0%|\x1b          \x1b| 0/4 [00:00<?, ?percent/s]\rPulling and hashing images...:  50%|\x1b\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88     \x1b| 2/4 [00:00<00:00,  8.33percent/s]\rPulling and hashing images...: 100%|\x1b\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\x1b| 4/4 [00:08<00:00,  2.61s/percent]\rPulling and hashing images...: 100%|\x1b\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\

In [16]:

# get the hash of the security policy
with open("template_kms_skr.json", "r") as f:
    # open the template and grab the cce policy
    template = json.loads(f.read())
    security_policy = template.get('resources')[0]['properties']['confidentialComputeProperties']['ccePolicy']
    # decode the base64 encoded policy and hash it
    sha256_hash_sum = sha256(base64.b64decode(security_policy)).hexdigest()
    # print the hash
    print("hash of security policy: ", sha256_hash_sum)


hash of security policy:  708927e5febd5666d4cac83e7cb76b2313c8b85e99626005e1a981486c3aa34f


### Step 4 

Deploy the container group to ACI.

In [17]:
#deploy ARM Template 
# TODO : Need to update the resource group name to your own resource group name
result = subprocess.run(["az deployment group create -g cc_aci -f template_kms_skr.json"], capture_output=True, shell=True)
result

CompletedProcess(args=['az deployment group create -g cc_aci -f template_kms_skr.json'], returncode=0, stdout=b'{\n  "id": "/subscriptions/7d862a03-c825-4759-9229-1333b52b7e29/resourceGroups/cc_aci/providers/Microsoft.Resources/deployments/template_kms_skr",\n  "location": null,\n  "name": "template_kms_skr",\n  "properties": {\n    "correlationId": "3c79ac76-c806-4898-ab86-8b09c5615351",\n    "debugSetting": null,\n    "dependencies": [],\n    "duration": "PT1M22.0059701S",\n    "error": null,\n    "mode": "Incremental",\n    "onErrorDeployment": null,\n    "outputResources": [\n      {\n        "id": "/subscriptions/7d862a03-c825-4759-9229-1333b52b7e29/resourceGroups/cc_aci/providers/Microsoft.ContainerInstance/containerGroups/attested_kms",\n        "resourceGroup": "cc_aci"\n      }\n    ],\n    "outputs": {\n      "containerIPv4Address": {\n        "type": "String",\n        "value": "20.75.173.14"\n      }\n    },\n    "parameters": {\n      "containerGroupName": {\n        "type

### Step 5 : Check for successful deployment on Azure Portal and get attestation token
In this step we will check for the successful deployment and get the attestation token verified by MAA. We will compare the contents of the "x-ms-sevsnpvm-hostdata" claim and check whether it matches the policy hash from step 3

In [34]:

# TODO: update the public_ip_address to the public ip address of your deployed container group. You can obtain the ip address from azure portal.  

public_ip_address = '20.88.190.100'
# call the maa endpoint
maa_response = requests.post(f'http://{public_ip_address}/attest/maa', 
                            json={"runtime_data": runtime_data, "maa_endpoint": attestation_endpoint})
print("Maa Response: ", maa_response.json())


Maa Response:  {'result': '{"token":"eyJhbGciOiJSUzI1NiIsImprdSI6Imh0dHBzOi8vc2hhcmVkZXVzMi5ldXMyLnRlc3QuYXR0ZXN0LmF6dXJlLm5ldC9jZXJ0cyIsImtpZCI6IjNiZENZSmFiemZoSVNGdGIzSjh5dUVFU1p3dWZWN2hoaDA4TjNaZmxBdUU9IiwidHlwIjoiSldUIn0.eyJleHAiOjE2OTM0Nzc2NDQsImlhdCI6MTY5MzQ0ODg0NCwiaXNzIjoiaHR0cHM6Ly9zaGFyZWRldXMyLmV1czIudGVzdC5hdHRlc3QuYXp1cmUubmV0IiwianRpIjoiOGE0ZWYzZWJmYmIxNDMyYzE1YzY3NGU0OTg4MTNjM2YyZmUwYTQzODM3YTg3OGViYzk5NTE1NWZlMjE5MjBmNyIsIm5iZiI6MTY5MzQ0ODg0NCwibm9uY2UiOiI3NjU1NjA4MzgxOTY3NjgxOTUzIiwieC1tcy1hdHRlc3RhdGlvbi10eXBlIjoic2V2c25wdm0iLCJ4LW1zLWNvbXBsaWFuY2Utc3RhdHVzIjoiYXp1cmUtY29tcGxpYW50LXV2bSIsIngtbXMtcG9saWN5LWhhc2giOiI5TlkwVm5UUS1JaUJyaUJwbFZVcEZiY3pjRGFFQlV3c2lGWUF6SHVfZ2NvIiwieC1tcy1ydW50aW1lIjp7ImtleXMiOlt7ImUiOiJBUUFCIiwia2V5X29wcyI6WyJlbmNyeXB0Il0sImtpZCI6Ik52aGZ1cTJjQ0lPQUI4WFI0WGk5UHIwTlBfOUNlTXpXUUd0V19IQUx6X3ciLCJrdHkiOiJSU0EiLCJuIjoidjk2NVNSbXlwOHpiRzVlTkZ1RENtbWlTZWFIcHVqRzJiQ19rZUxTdXp2RE1MTzFXeXJVSnZlYWE1YnpNb08wcEE0NnBYa21icUhpc296VnpwaU5ETENvNmQzejRUckdNZUZ

In [33]:

token = json.loads(maa_response.json().get("result")).get("token")

# verify the token

header = jwt.get_unverified_header(token)
kid = header['kid']
kid


'3bdCYJabzfhISFtb3J8yuEESZwufV7hhh08N3ZflAuE='

In [32]:


# Find the key with a matching 'kid' in the JWKS
key_to_use = None
for key_kid, key in jwks:
        if key_kid == kid:
            key_to_use = key
            break

if key_to_use is not None:
        try:
            payload = jwt.decode(token, key=key_to_use, algorithms=["RS256"])
            print("Valid JWT : Attestation token signature verified:", payload)
            
        except jwt.InvalidTokenError:
            print("Invalid JWT")
else:
        print("No matching key found in JWKS")



Valid JWT : Attestation token signature verified: {'exp': 1693477605, 'iat': 1693448805, 'iss': 'https://sharedeus2.eus2.test.attest.azure.net', 'jti': '30389604158ca4e99a5b1e2d53698e5ec5fe46966aca468bea4b3c20a9eef724', 'nbf': 1693448805, 'nonce': '16103641454482022980', 'x-ms-attestation-type': 'sevsnpvm', 'x-ms-compliance-status': 'azure-compliant-uvm', 'x-ms-policy-hash': '9NY0VnTQ-IiBriBplVUpFbczcDaEBUwsiFYAzHu_gco', 'x-ms-runtime': {'keys': [{'e': 'AQAB', 'key_ops': ['encrypt'], 'kid': 'Nvhfuq2cCIOAB8XR4Xi9Pr0NP_9CeMzWQGtW_HALz_w', 'kty': 'RSA', 'n': 'v965SRmyp8zbG5eNFuDCmmiSeaHpujG2bC_keLSuzvDMLO1WyrUJveaa5bzMoO0pA46pXkmbqHisozVzpiNDLCo6d3z4TrGMeFPf2APIMu-RSrzN56qvHVyIr5caWfHWk-FMRDwAefyNYRHkdYYkgmFK44hhUdtlCAKEv5UQpFZjvh4iI9jVBdGYMyBaKQLhjI5WIh-QG6Za5sSuOCFMnmuyuvN5DflpLFz595Ss-EoBIY-Nil6lCtvcGgR-IbjUYHAOs5ajamTzgeO8kx3VCE9HcyKmyUZsiyiF6IDRp2Bpy3NHTjIz7tmkpTHx7tHnRtlfE2FUv0B6i_QYl_ZA5Q'}]}, 'x-ms-sevsnpvm-authorkeydigest': '0000000000000000000000000000000000000000000000000000000

In [21]:

print("SEV-SNP Host Data:\n", payload.get("x-ms-sevsnpvm-hostdata"))
if(sha256_hash_sum == payload.get("x-ms-sevsnpvm-hostdata")):
    print("Security Policy Hash Matches")
    print("Host is Trusted")
else:
    print("Security Policy Hash Does Not Match")
    print("Host is Not Trusted")

SEV-SNP Host Data:
 708927e5febd5666d4cac83e7cb76b2313c8b85e99626005e1a981486c3aa34f
Security Policy Hash Matches
Host is Trusted


### Step 6 : Error scenario : Attempt to deploy an image different than what is captured in the security policy 
We want to break the deployment by deploying a different image, in this case a container image that calculates the product of two numbers instead of the sum with the same repository and tag.

After the successful execution of this step, please go to the azure portal and restart the container group that was deployed previously in Step 4. Once the container group restarts, the new image will break the policy and fail the deployment as the updated image does not reflect the policy. This is simulating an attack scenario where the container image is updated by a malicious actor. 
This is also why the `latest` tag is not recommended for Confidential ACI. 


In [40]:
# deploy product + skr container with ARM Template from CLI
#cacidemo is a sample repository name, you can change it to something else and update below. For this scenario the name should be the same as the one used in step2
import subprocess
subprocess.run(["docker build -t tanaybaswa/attestedkms:latest /home/tanaybaswa/code/azure_coco_aci/kms_skr/nginx/"], capture_output=True, shell=True)
subprocess.run(["docker push tanaybaswa/attestedkms:latest"], capture_output=True, shell=True)

CompletedProcess(args=['docker push tanaybaswa/attestedkms:latest'], returncode=0, stdout=b'The push refers to repository [docker.io/tanaybaswa/attestedkms]\n563c64030925: Preparing\n6fb960878295: Preparing\ne161c3f476b5: Preparing\n8a7e12012e6f: Preparing\nd0a62f56ef41: Preparing\n4713cb24eeff: Preparing\n511780f88f80: Preparing\n4713cb24eeff: Waiting\n511780f88f80: Waiting\nd0a62f56ef41: Layer already exists\n6fb960878295: Layer already exists\ne161c3f476b5: Layer already exists\n563c64030925: Layer already exists\n8a7e12012e6f: Layer already exists\n511780f88f80: Layer already exists\n4713cb24eeff: Layer already exists\nlatest: digest: sha256:8a51b2bbb7a030e998ddfd4cb0669f8d0479512a245be12176f13dcdd115aa55 size: 1778\n', stderr=b'')

### Step 7: Revert back to Correct Image

In [41]:
# build and push the image to the registry
subprocess.run(["docker build -t tanaybaswa/attestedkms:latest /home/tanaybaswa/code/enkryptai/key-manager/"], shell=True, capture_output=True, text=True)
subprocess.run(["docker push tanaybaswa/attestedkms:latest"], shell=True, capture_output=True, text=True)

CompletedProcess(args=['docker push tanaybaswa/attestedkms:latest'], returncode=0, stdout='The push refers to repository [docker.io/tanaybaswa/attestedkms]\n5f70bf18a086: Preparing\nf73ab0be9222: Preparing\nf2b087a0b61e: Preparing\nbd6143418689: Preparing\n1847aa24cd34: Preparing\n2f6e140f1970: Preparing\nf30c9841a094: Preparing\na886fdc1df20: Preparing\nb2e5b1eee192: Preparing\nb485c6cd33a6: Preparing\n6aa872026017: Preparing\n43ba18a5eaf8: Preparing\nff61a9b258e5: Preparing\nb2e5b1eee192: Waiting\nb485c6cd33a6: Waiting\n6aa872026017: Waiting\n43ba18a5eaf8: Waiting\nf30c9841a094: Waiting\nff61a9b258e5: Waiting\na886fdc1df20: Waiting\n2f6e140f1970: Waiting\n5f70bf18a086: Layer already exists\nbd6143418689: Layer already exists\nf73ab0be9222: Layer already exists\nf2b087a0b61e: Layer already exists\n1847aa24cd34: Layer already exists\n2f6e140f1970: Layer already exists\nf30c9841a094: Layer already exists\nb2e5b1eee192: Layer already exists\na886fdc1df20: Layer already exists\nb485c6cd33

### Step 8 : Restart the Container Group Again, Go Through checks and check Attestation.

In [42]:
public_ip_address = '20.242.201.73'
# call the maa endpoint
maa_response = requests.post(f'http://{public_ip_address}/attest/maa', 
                            json={"runtime_data": runtime_data, "maa_endpoint": attestation_endpoint})
print("Maa Response: ", maa_response.json())

token = json.loads(maa_response.json().get("result")).get("token")

# verify the token

header = jwt.get_unverified_header(token)
kid = header['kid']
kid

Maa Response:  {'result': '{"token":"eyJhbGciOiJSUzI1NiIsImprdSI6Imh0dHBzOi8vc2hhcmVkZXVzMi5ldXMyLnRlc3QuYXR0ZXN0LmF6dXJlLm5ldC9jZXJ0cyIsImtpZCI6IjNiZENZSmFiemZoSVNGdGIzSjh5dUVFU1p3dWZWN2hoaDA4TjNaZmxBdUU9IiwidHlwIjoiSldUIn0.eyJleHAiOjE2OTM0NzgwNzUsImlhdCI6MTY5MzQ0OTI3NSwiaXNzIjoiaHR0cHM6Ly9zaGFyZWRldXMyLmV1czIudGVzdC5hdHRlc3QuYXp1cmUubmV0IiwianRpIjoiMmJlZTdhMDY3ZTRkZDM4NDZmMDJmZDRiNTBlMjI5NDdhYzNmZjEyZTFmZGMwZTI4ZTlhZGFlN2NkN2M2YTcwZSIsIm5iZiI6MTY5MzQ0OTI3NSwibm9uY2UiOiIxNzQ1NDg1ODgyMTk5NzE2NDM1MiIsIngtbXMtYXR0ZXN0YXRpb24tdHlwZSI6InNldnNucHZtIiwieC1tcy1jb21wbGlhbmNlLXN0YXR1cyI6ImF6dXJlLWNvbXBsaWFudC11dm0iLCJ4LW1zLXBvbGljeS1oYXNoIjoiOU5ZMFZuVFEtSWlCcmlCcGxWVXBGYmN6Y0RhRUJVd3NpRllBekh1X2djbyIsIngtbXMtcnVudGltZSI6eyJrZXlzIjpbeyJlIjoiQVFBQiIsImtleV9vcHMiOlsiZW5jcnlwdCJdLCJraWQiOiJOdmhmdXEyY0NJT0FCOFhSNFhpOVByME5QXzlDZU16V1FHdFdfSEFMel93Iiwia3R5IjoiUlNBIiwibiI6InY5NjVTUm15cDh6Ykc1ZU5GdURDbW1pU2VhSHB1akcyYkNfa2VMU3V6dkRNTE8xV3lyVUp2ZWFhNWJ6TW9PMHBBNDZwWGttYnFIaXNvelZ6cGlORExDbzZkM3o0VHJHTWV

'3bdCYJabzfhISFtb3J8yuEESZwufV7hhh08N3ZflAuE='

In [43]:

# Find the key with a matching 'kid' in the JWKS
key_to_use = None
for key_kid, key in jwks:
        if key_kid == kid:
            key_to_use = key
            break
if key_to_use is not None:
        try:
            payload = jwt.decode(token, key=key_to_use, algorithms=["RS256"])
            print("Valid JWT : Attestation token signature verified:", payload)
            
        except jwt.InvalidTokenError:
            print("Invalid JWT")
else:
        print("No matching key found in JWKS")


print("SEV-SNP Host Data:\n", payload.get("x-ms-sevsnpvm-hostdata"))
if(sha256_hash_sum == payload.get("x-ms-sevsnpvm-hostdata")):
    print("Security Policy Hash Matches")
    print("Host is Trusted")
else:
    print("Security Policy Hash Does Not Match")
    print("Host is Not Trusted")


Valid JWT : Attestation token signature verified: {'exp': 1693478075, 'iat': 1693449275, 'iss': 'https://sharedeus2.eus2.test.attest.azure.net', 'jti': '2bee7a067e4dd3846f02fd4b50e22947ac3ff12e1fdc0e28e9adae7cd7c6a70e', 'nbf': 1693449275, 'nonce': '17454858821997164352', 'x-ms-attestation-type': 'sevsnpvm', 'x-ms-compliance-status': 'azure-compliant-uvm', 'x-ms-policy-hash': '9NY0VnTQ-IiBriBplVUpFbczcDaEBUwsiFYAzHu_gco', 'x-ms-runtime': {'keys': [{'e': 'AQAB', 'key_ops': ['encrypt'], 'kid': 'Nvhfuq2cCIOAB8XR4Xi9Pr0NP_9CeMzWQGtW_HALz_w', 'kty': 'RSA', 'n': 'v965SRmyp8zbG5eNFuDCmmiSeaHpujG2bC_keLSuzvDMLO1WyrUJveaa5bzMoO0pA46pXkmbqHisozVzpiNDLCo6d3z4TrGMeFPf2APIMu-RSrzN56qvHVyIr5caWfHWk-FMRDwAefyNYRHkdYYkgmFK44hhUdtlCAKEv5UQpFZjvh4iI9jVBdGYMyBaKQLhjI5WIh-QG6Za5sSuOCFMnmuyuvN5DflpLFz595Ss-EoBIY-Nil6lCtvcGgR-IbjUYHAOs5ajamTzgeO8kx3VCE9HcyKmyUZsiyiF6IDRp2Bpy3NHTjIz7tmkpTHx7tHnRtlfE2FUv0B6i_QYl_ZA5Q'}]}, 'x-ms-sevsnpvm-authorkeydigest': '0000000000000000000000000000000000000000000000000000000